In [1]:
#https://shotanuki.com/python%E3%81%A7pdf%E3%82%92%E7%94%BB%E5%83%8F%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%99%E3%82%8B/
#PythonでPDFを画像に変換する
from pdf2image import convert_from_path
import datetime
import os
now = datetime.date.today()

# カレントディレクトリの全画像をループする
for pdfname in os.listdir('./mergedTRF/'):# ❶
    ext = os.path.splitext(pdfname)
    #if ext == '.pdf':
    images = convert_from_path('./mergedTRF/'+pdfname)
    os.makedirs('./trf/'+str(now), exist_ok=True)
    i = 0
    for image in images:
        image.save('./trf/'+str(now) +'/trf_' + str(pdfname)+'_'+str(i)+'.png', 'png')
        i += 1
       

In [2]:
'''
nums_old = [
    #(650,730,50,450), #HspID
    #(655,735,1025,1115),#DOB MM
    #(655,735,1155,1245),#DOB DD
    #(655,735,1320,1410),#DOB YY

    #(860,940,440,530), #DD MM
    #(860,940,555,645), #DD DD
    #(860,940,725,815), #DD YY
    (1055,1135,285,375), #EDC MM Old
    (1055,1135,415,505), #EDC DD Old
    (1055,1135,585,675), #EDC YY  DD NewOld
    (1320,1410,595,685), #Donor YY NewOld
    (1320,1410,770,860), #Donor MM NewOld
    (1540,1630,425,545), #Weight
    ]
nums = [
    #(650,730,50,450), #HspID
    #(655,735,1025,1115),#DOB MM
    #(655,735,1155,1245),#DOB DD
    #(655,735,1320,1410),#DOB YY
    #(860,940,440,530), #DD MM
    #(860,940,555,645), #DD DD
    #(860,940,725,815), #DD YY
    (1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
'''
%matplotlib inline
from keras.models import load_model
import numpy as np
import cv2
import csv
import matplotlib.pyplot as plt
from PIL import Image
import sys,os
import datetime
now = datetime.date.today()
now_time = datetime.datetime.today()
now_time = str(now_time.hour)+'-'+str(now_time.minute)+'-'+str(now_time.second)

os.makedirs('./trf/'+str(now), exist_ok=True)
os.makedirs('./number/'+ str(now), exist_ok = True)
os.makedirs('./number/' + str(now) + '/10', exist_ok = True)
os.makedirs('./number/' + str(now) + '/0', exist_ok = True)
os.makedirs('./number/' + str(now) + '/1', exist_ok = True)
os.makedirs('./number/' + str(now) + '/2', exist_ok = True)
os.makedirs('./number/' + str(now) + '/3', exist_ok = True)
os.makedirs('./number/' + str(now) + '/4', exist_ok = True)
os.makedirs('./number/' + str(now) + '/5', exist_ok = True)
os.makedirs('./number/' + str(now) + '/6', exist_ok = True)
os.makedirs('./number/' + str(now) + '/7', exist_ok = True)
os.makedirs('./number/' + str(now) + '/8', exist_ok = True)
os.makedirs('./number/' + str(now) + '/9', exist_ok = True)
os.makedirs('./results/number_results/' + str(now), exist_ok = True)
os.makedirs('./csv/'+ str(now), exist_ok = True)
path_w = './csv/'+ str(now)+'/'+str(now)+'_'+str(now_time)+'.csv'

labels = ["0","1","2","3","4","5","6","7","8","9","Error"]
model = load_model('CNN-number-model.h5')
model.load_weights('CNN-number-weight.h5')

labels1 = ["Error ", "N ", "Y "]
model1 = load_model('CNN-mk-model.h5')
model1.load_weights('CNN-mk-weight.h5')

Using TensorFlow backend.


In [3]:
# 依頼書画像から番号領域を抽出する関数
def detect_DOB_MM(fname):
    # 画像を読み込む
    nums = [
    #(650,730,50,450), #HspID
    (655,735,1025,1115),#DOB MM
    (655,735,1155,1245),#DOB DD
    (655,735,1320,1410),#DOB YY
    (860,940,440,530), #DD MM
    (860,940,555,645), #DD DD
    (860,940,725,815), #DD YY
    (1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    for j, pt in enumerate(nums):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
            

In [4]:
# 依頼書画像から番号領域を抽出する関数
def detect_HspID(fname):
    # 画像を読み込む
    nums = [
    (650,730,50,450), #HspID
    (655,735,1025,1115),#DOB MM
    (655,735,1155,1245),#DOB DD
    (655,735,1320,1410),#DOB YY
    (860,940,440,530), #DD MM
    (860,940,555,645), #DD DD
    (860,940,725,815), #DD YY
    (1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    for j, pt in enumerate(nums):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
            

In [5]:
# 依頼書画像から番号領域を抽出する関数
def detect_DOB_DD(fname):
    # 画像を読み込む
    nums = [
    #(650,730,50,450), #HspID
    #(655,735,1025,1115),#DOB MM
    (655,735,1155,1245),#DOB DD
    (655,735,1320,1410),#DOB YY
    (860,940,440,530), #DD MM
    (860,940,555,645), #DD DD
    (860,940,725,815), #DD YY
    (1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    for j, pt in enumerate(nums):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
     

In [6]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DOB_YY(fname):
    # 画像を読み込む
    nums = [
    #(650,730,50,450), #HspID
    #(655,735,1025,1115),#DOB MM
    #(655,735,1155,1245),#DOB DD
    (655,735,1320,1410),#DOB YY
    (860,940,440,530), #DD MM
    (860,940,555,645), #DD DD
    (860,940,725,815), #DD YY
    (1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    for j, pt in enumerate(nums):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
     

In [7]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DD_MM(fname):
    # 画像を読み込む
    nums = [
    #(650,730,50,450), #HspID
    #(655,735,1025,1115),#DOB MM
    #(655,735,1155,1245),#DOB DD
    #(655,735,1320,1410),#DOB YY
    (860,940,440,530), #DD MM
    (860,940,555,645), #DD DD
    (860,940,725,815), #DD YY
    (1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    for j, pt in enumerate(nums):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
    

In [8]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DD_DD(fname):
    # 画像を読み込む
    nums = [
    #(650,730,50,450), #HspID
    #(655,735,1025,1115),#DOB MM
    #(655,735,1155,1245),#DOB DD
    #(655,735,1320,1410),#DOB YY
    #(860,940,440,530), #DD MM
    (860,940,555,645), #DD DD
    (860,940,725,815), #DD YY
    (1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    for j, pt in enumerate(nums):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 

In [9]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DD_YY(fname):
    # 画像を読み込む
    nums = [
    #(650,730,50,450), #HspID
    #(655,735,1025,1115),#DOB MM
    #(655,735,1155,1245),#DOB DD
    #(655,735,1320,1410),#DOB YY
    #(860,940,440,530), #DD MM
    #(860,940,555,645), #DD DD
    (860,940,725,815), #DD YY
    (1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    for j, pt in enumerate(nums):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
       

In [10]:
'''# 依頼書画像から番号領域を抽出する関数
def detect_DD_YY(fname):
    # 画像を読み込む
    nums_old = [
    #(650,730,50,450), #HspID
    #(655,735,1025,1115),#DOB MM
    #(655,735,1155,1245),#DOB DD
    #(655,735,1320,1410),#DOB YY

    #(860,940,440,530), #DD MM
    #(860,940,555,645), #DD DD
    (860,940,725,815), #DD YY
    (1055,1135,285,375), #EDC MM Old
    (1055,1135,415,505), #EDC DD Old
    (1055,1135,585,675), #EDC YY  DD NewOld
    (1320,1410,595,685), #Donor YY NewOld
    (1320,1410,770,860), #Donor MM NewOld
    (1540,1630,425,545), #Weight
    ]
    nums = [
    #(650,730,50,450), #HspID
    #(655,735,1025,1115),#DOB MM
    #(655,735,1155,1245),#DOB DD
    #(655,735,1320,1410),#DOB YY
    #(860,940,440,530), #DD MM
    #(860,940,555,645), #DD DD
    (860,940,725,815), #DD YY
    (1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(nums_old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(nums):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not(10< w < 50): continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 10: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            '''

'# 依頼書画像から番号領域を抽出する関数\ndef detect_DD_YY(fname):\n    # 画像を読み込む\n    nums_old = [\n    #(650,730,50,450), #HspID\n    #(655,735,1025,1115),#DOB MM\n    #(655,735,1155,1245),#DOB DD\n    #(655,735,1320,1410),#DOB YY\n\n    #(860,940,440,530), #DD MM\n    #(860,940,555,645), #DD DD\n    (860,940,725,815), #DD YY\n    (1055,1135,285,375), #EDC MM Old\n    (1055,1135,415,505), #EDC DD Old\n    (1055,1135,585,675), #EDC YY  DD NewOld\n    (1320,1410,595,685), #Donor YY NewOld\n    (1320,1410,770,860), #Donor MM NewOld\n    (1540,1630,425,545), #Weight\n    ]\n    nums = [\n    #(650,730,50,450), #HspID\n    #(655,735,1025,1115),#DOB MM\n    #(655,735,1155,1245),#DOB DD\n    #(655,735,1320,1410),#DOB YY\n    #(860,940,440,530), #DD MM\n    #(860,940,555,645), #DD DD\n    (860,940,725,815), #DD YY\n    (1055,1135,185,275), #EDC MM New\n    (1055,1135,295,385), #EDC DD New\n    (1055,1135,475,565), #EDC YY  DD NewOld\n    (1370,1450,555,645), #Donor YY NewOld\n    (1370,1450,695,785), #Donor MM

In [11]:

# 依頼書画像から番号領域を抽出する関数
def detect_EDC_MM(fname):
    # 画像を読み込む
    nums_old = [
    (1055,1135,285,375), #EDC MM Old
    (1055,1135,415,505), #EDC DD Old
    (1055,1135,585,675), #EDC YY  DD NewOld
    (1320,1410,595,685), #Donor YY NewOld
    (1320,1410,770,860), #Donor MM NewOld
    (1540,1630,425,545), #Weight
    ]
    nums = [
    (1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(nums_old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(nums):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not(10< w < 50): continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 10: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            

In [12]:
# 依頼書画像から番号領域を抽出する関数
def detect_EDC_DD(fname):
    nums_old = [
    #(1055,1135,285,375), #EDC MM Old
    (1055,1135,415,505), #EDC DD Old
    (1055,1135,585,675), #EDC YY  DD NewOld
    (1320,1410,595,685), #Donor YY NewOld
    (1320,1410,770,860), #Donor MM NewOld
    (1540,1630,425,545), #Weight
    ]
    nums = [
    #(1055,1135,185,275), #EDC MM New
    (1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    # 画像を読み込む
    
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(nums_old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(nums):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not(10< w < 50): continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 10: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            

In [13]:

def detect_EDC_YY(fname):
    # 画像を読み込む
    nums_old = [
    #(1055,1135,285,375), #EDC MM Old
    #(1055,1135,415,505), #EDC DD Old
    (1055,1135,585,675), #EDC YY  DD NewOld
    (1320,1410,595,685), #Donor YY NewOld
    (1320,1410,770,860), #Donor MM NewOld
    (1540,1630,425,545), #Weight
    ]
    nums = [
    #(1055,1135,185,275), #EDC MM New
    #(1055,1135,295,385), #EDC DD New
    (1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(nums_old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(nums):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not(10< w < 50): continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 10: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            

In [14]:

def detect_Donor_YY(fname):
    # 画像を読み込む
    nums_old = [
    #(1055,1135,285,375), #EDC MM Old
    #(1055,1135,415,505), #EDC DD Old
    #(1055,1135,585,675), #EDC YY  DD NewOld
    (1320,1410,595,685), #Donor YY NewOld
    (1320,1410,770,860), #Donor MM NewOld
    (1540,1630,425,545), #Weight
    ]
    nums = [
    #(1055,1135,185,275), #EDC MM New
    #(1055,1135,295,385), #EDC DD New
    #(1055,1135,475,565), #EDC YY  DD NewOld
    (1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(nums_old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(nums):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not(10< w < 50): continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 10: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            

In [15]:
def detect_Donor_MM(fname):
    # 画像を読み込む
    nums_old = [
    #(1055,1135,285,375), #EDC MM Old
    #(1055,1135,415,505), #EDC DD Old
    #(1055,1135,585,675), #EDC YY  DD NewOld
    #(1320,1410,595,685), #Donor YY NewOld
    (1320,1410,770,860), #Donor MM NewOld
    (1540,1630,425,545), #Weight
    ]
    nums = [
    #(1055,1135,185,275), #EDC MM New
    #(1055,1135,295,385), #EDC DD New
    #(1055,1135,475,565), #EDC YY  DD NewOld
    #(1370,1450,555,645), #Donor YY NewOld
    (1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(nums_old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(nums):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not(10< w < 50): continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 10: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            

In [16]:
def detect_Wt(fname):
    # 画像を読み込む
    nums_old = [
    #(1055,1135,285,375), #EDC MM Old
    #(1055,1135,415,505), #EDC DD Old
    #(1055,1135,585,675), #EDC YY  DD NewOld
    #(1320,1410,595,685), #Donor YY NewOld
    #(1320,1410,770,860), #Donor MM NewOld
    (1540,1630,425,545), #Weight
    ]
    nums = [
    #(1055,1135,185,275), #EDC MM New
    #(1055,1135,295,385), #EDC DD New
    #(1055,1135,475,565), #EDC YY  DD NewOld
    #(1370,1450,555,645), #Donor YY NewOld
    #(1370,1450,695,785), #Donor MM NewOld
    (1540,1630,425,545), #Weight    
    ]
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(nums_old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(10< w < 50): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(nums):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not(10< w < 50): continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 10: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 

In [17]:
labels = ["0","1","2","3","4","5","6","7","8","9","Error"]
model = load_model('CNN-number-model.h5')
model.load_weights('CNN-number-weight.h5')

for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_HspID('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'HspID'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/HspID_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/HspID_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [18]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DOB_MM('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DOB_MM'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DOB_MM_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DOB_MM_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [19]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DOB_DD('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DOB_DD'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DOB_DD_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DOB_DD_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')
                    

In [20]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DD_MM('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DD_MM'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DD_MM_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DD_MM_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')
                    

In [21]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DD_DD('./trf/'+str(now)+'/'+ filename)
            if img is None : break
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DD_DD'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DD_DD_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DD_DD_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [22]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DD_YY('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DD_YY'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DD_YY_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DD_YY_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [23]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_EDC_DD('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'EDC_DD'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/EDC_DD_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/EDC_DD_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [24]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_EDC_MM('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'EDC_MM'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/EDC_MM_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/EDC_MM_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [25]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_EDC_YY('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'EDC_YY'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/EDC_YY_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/EDC_YY_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [26]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_Donor_YY('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'Donor_YY'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/Donor_YY_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/Donor_YY_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [27]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_Donor_MM('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'Donor_MM'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/Donor_MM_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/Donor_MM_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [28]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_Wt('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'Wt'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/Wt_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/Wt_'+ str(j)+'_'+ filename, img)
                if idx == 10: break
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx])+'\t')
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [29]:
import shutil
shutil.move('./number/' + str(now)+'/','./number/'+str(now)+'_'+ str(now_time)+'/')
shutil.move('./results/number_results/' + str(now) + '/','./results/number_results/' + str(now) + '_'+str(now_time)+'/')
shutil.move('./trf/'+str(now)+'/','./finished_trf/number/'+str(now)+'_'+str(now_time)+'/')

'./finished_trf/number/2019-08-28_13-55-0/'